In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# read 'coefficients.csv'
coefficients_df = pd.read_csv('coefficients.csv', index_col=0)

# date, a, b, c, risk_free_rate
print(coefficients_df.head())

                 a       b       c  risk_free_rate
date                                              
2019-01-31  0.3432  0.0695  0.0000            0.02
2019-02-28  0.4540  0.0718  0.0000            0.02
2019-03-31  0.4245  0.0616  0.0119            0.02
2019-04-30  0.3623  0.0753  0.0355            0.02
2019-05-31  0.1075  0.0661  0.0548            0.02


In [6]:
class LSTM(nn.Module):
    def __init__(self, input_size=4, hidden_layer_size=100, output_size=4):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq), 1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

In [7]:
model = LSTM()
loss_function = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Prepare the data
data = coefficients_df[['a', 'b', 'c', 'risk_free_rate']].values
data = torch.tensor(data, dtype=torch.float32)



In [8]:
# train the model
epochs = 150

for i in range(epochs):
    for index in range(len(data)):
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                             torch.zeros(1, 1, model.hidden_layer_size))

        input_tensor = data[index].unsqueeze(0)  # Add batch dimension
        y_pred = model(input_tensor)
        y = data[index]
        single_loss = loss_function(y_pred, y)
        single_loss.backward()
        optimizer.step()

    if i % 10 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

epoch:   1 loss: 0.00234932
epoch:  11 loss: 0.00001984
epoch:  21 loss: 0.00005393
epoch:  31 loss: 0.00005855
epoch:  41 loss: 0.00005259
epoch:  51 loss: 0.00009873
epoch:  61 loss: 0.00014173
epoch:  71 loss: 0.00019541
epoch:  81 loss: 0.00022502
epoch:  91 loss: 0.00018914
epoch: 101 loss: 0.00014970
epoch: 111 loss: 0.00013757
epoch: 121 loss: 0.00019696
epoch: 131 loss: 0.00018268
epoch: 141 loss: 0.00010552
